In [1]:
import re
import spacy
import gensim
import logging
import pandas as pd
import numpy as np
# remove warnings
logging.basicConfig(level=logging.ERROR)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# lectura de la data y eliminación de registros repetidos y nulos
documents = pd.read_csv('./datos/simpsons_dataset.csv').dropna().drop_duplicates()
documents = documents.reset_index(drop=True)
print(documents.shape)
display(documents.head())

(126646, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
# Filtrado por personajes principales
main_characters =  ["Lisa Simpson", "Bart Simpson", "Homer Simpson", "Marge Simpson"]
documents = documents[documents["raw_character_text"].isin(main_characters)]
print(documents.shape)
display(documents.head())

(60610, 2)


,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
7,Bart Simpson,Victory party under the slide!
8,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Lisa Simpson,Do you know where I could find him?


In [4]:
def get_sublist(list:list, n:int) -> list:
    """
    Obtener sublistas de una lista
    """
    return [list[i:i+n] for i in range(0, len(list), n)]


In [5]:
def get_vectors_characters(characters: pd.DataFrame) -> pd.DataFrame:
    """
    Extraer dialogos para todos los personajes en grupos de 5 oraciones
    """
    sentences_big = get_sublist(characters['spoken_words'].tolist(),5)
    df = pd.DataFrame({ 'spoken_words': sentences_big})
    for index,dialogue in df['spoken_words'].items():
        df.loc[index,"spoken_words_concatenated"] = ' '.join(dialogue)
    return df

In [6]:
documents_dialogue_concatenated = documents.groupby("raw_character_text")  \
                                    .apply(lambda x: get_vectors_characters(x)) \
                                    .reset_index("raw_character_text")

In [7]:
def convert_label(label:str) -> int:
    """ 
    Categoriza los labels
    """
    if label == "bart simpson":
        return 1
    elif label == "lisa simpson":
        return 2
    elif label == "homer simpson":
        return 3
    else : 
        return 4
   

In [8]:
documents_dialogue_concatenated["raw_character_text"] = documents_dialogue_concatenated["raw_character_text"].apply(lambda x: x.lower())
documents_dialogue_concatenated["raw_character_text"] = documents_dialogue_concatenated["raw_character_text"].apply(convert_label)
documents_dialogue_concatenated["spoken_words_concatenated"] = documents_dialogue_concatenated["spoken_words_concatenated"].apply(lambda x: x.lower())
documents_dialogue_concatenated.head()

,raw_character_text,spoken_words,spoken_words_concatenated
0,1,"[Victory party under the slide!, Hey, thanks f...","victory party under the slide! hey, thanks for..."
1,1,"[Somebody must have voted., Uh oh., I demand a...",somebody must have voted. uh oh. i demand a re...
2,1,"[Ah, Dad, if just me, Milhouse and Lewis had v...","ah, dad, if just me, milhouse and lewis had vo..."
3,1,"[Please Dad., What?, Yes sir., They're fightin...",please dad. what? yes sir. they're fighting in...
4,1,"[Dad, I have as much respect for you as I ever...","dad, i have as much respect for you as i ever ..."


In [9]:

sentences = documents_dialogue_concatenated['spoken_words_concatenated'].values
y = documents_dialogue_concatenated['raw_character_text'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.2, random_state=1)

y_train_categorical = tf.one_hot(y_train,4)
y_test_categorical = tf.one_hot(y_test,4)


train_dataset = tf.data.Dataset.from_tensor_slices((sentences_train,y_train_categorical))
test_dataset = tf.data.Dataset.from_tensor_slices((sentences_test,y_test_categorical))

text_dataset = tf.data.Dataset.from_tensor_slices((sentences_train))

In [10]:

print('Number of rows in the total set: {}'.format(sentences.shape[0]))
print('Number of rows of bart class: {}'.format(y[y == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y[y == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y[y == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y[y == 3].shape[0]))
print('---------------------------------------------')

print('Number of rows in the training set: {}'.format(sentences_train.shape[0]))
print('Number of rows of bart class: {}'.format(y_train[y_train == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y_train[y_train == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y_train[y_train == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y_train[y_train == 3].shape[0]))
print('---------------------------------------------')

print('Number of rows in the test set: {}'.format(sentences_test.shape[0]))

print('Number of rows of bart class: {}'.format(y_test[y_test == 0].shape[0]))
print('Number of rows of lisa class: {}'.format(y_test[y_test == 1].shape[0]))
print('Number of rows of homer class: {}'.format(y_test[y_test == 2].shape[0]))
print('Number of rows of marge class: {}'.format(y_test[y_test == 3].shape[0]))


Number of rows in the total set: 12123
Number of rows of bart class: 0
Number of rows of lisa class: 2414
Number of rows of homer class: 2029
Number of rows of marge class: 5195
---------------------------------------------
Number of rows in the training set: 9698
Number of rows of bart class: 0
Number of rows of lisa class: 1913
Number of rows of homer class: 1598
Number of rows of marge class: 4171
---------------------------------------------
Number of rows in the test set: 2425
Number of rows of bart class: 0
Number of rows of lisa class: 501
Number of rows of homer class: 431
Number of rows of marge class: 1024


## Vectorize Layer Binary

In [11]:
vectorize_layer_binary = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='binary', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_binary.adapt(text_dataset)


In [12]:
vocab_size = len(vectorize_layer_binary.get_vocabulary())
vocab_size

200

In [13]:
embedding_dim = 10

In [14]:
modelsequ1 = Sequential()
modelsequ1.add(vectorize_layer_binary)
modelsequ1.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ1.add(layers.GlobalAveragePooling1D())
modelsequ1.add(layers.Dense(50, activation='relu', name="Hidden_layer_2"))
modelsequ1.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse']) 


In [15]:
batch_size = 100
epochs = 10

# Fit the model using the train and test datasets.
modelsequ1.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 3s 16ms/step - loss: 1.0135 - acc: 0.4211 - mse: 0.1511 - val_loss: 0.9472 - val_acc: 0.4223 - val_mse: 0.1443
Epoch 2/10
97/97 [==============================] - 1s 11ms/step - loss: 0.8712 - acc: 0.4301 - mse: 0.1395 - val_loss: 0.8648 - val_acc: 0.4223 - val_mse: 0.1422
Epoch 3/10
97/97 [==============================] - 1s 11ms/step - loss: 0.8197 - acc: 0.4301 - mse: 0.1390 - val_loss: 0.8389 - val_acc: 0.4223 - val_mse: 0.1422
Epoch 4/10
97/97 [==============================] - 1s 11ms/step - loss: 0.8092 - acc: 0.4301 - mse: 0.1390 - val_loss: 0.8348 - val_acc: 0.4223 - val_mse: 0.1418
Epoch 5/10
97/97 [==============================] - 1s 10ms/step - loss: 0.8071 - acc: 0.4301 - mse: 0.1391 - val_loss: 0.8346 - val_acc: 0.4223 - val_mse: 0.1413
Epoch 6/10
97/97 [==============================] - 1s 10ms/step - loss: 0.8065 - acc: 0.4301 - mse: 0.1394 - val_loss: 0.8352 - val_acc: 0.4223 - val_mse: 0.1414
Epoch 7/10
97/97 [====

In [16]:
modelsequ1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 200, 10)           2000      
                                                                 
 global_average_pooling1d (G  (None, 10)               0         
 lobalAveragePooling1D)                                          
                                                                 
 Hidden_layer_2 (Dense)      (None, 50)                550       
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 2,754
Trainable params: 2,754
Non-trainabl

In [17]:
modelsequ1.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 1s 4ms/step - loss: 0.8386 - acc: 0.4223 - mse: 0.1440


[0.8386245369911194, 0.4222680330276489, 0.14397282898426056]

In [18]:
pred = modelsequ1.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize Layer count

In [19]:
vectorize_layer_count = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='count', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer_count.adapt(text_dataset)

In [20]:
embedding_dim = 10
vocab_size = len(vectorize_layer_count.get_vocabulary())
vocab_size

200

In [21]:
modelsequ2 = Sequential()
modelsequ2.add(vectorize_layer_count)
modelsequ2.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ2.add(layers.GlobalAveragePooling1D())
modelsequ2.add(layers.Dense(100, activation='relu', name="Hidden_layer"))
modelsequ2.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 


In [22]:
# Fit the model using the train and test datasets.
batch_size = 100
epochs = 10

modelsequ2.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 5s 35ms/step - loss: 1.0301 - acc: 0.4301 - mse: 0.1531 - val_loss: 1.0035 - val_acc: 0.4223 - val_mse: 0.1493
Epoch 2/10
97/97 [==============================] - 3s 27ms/step - loss: 0.9474 - acc: 0.4301 - mse: 0.1436 - val_loss: 0.9475 - val_acc: 0.4223 - val_mse: 0.1439
Epoch 3/10
97/97 [==============================] - 3s 27ms/step - loss: 0.9038 - acc: 0.4301 - mse: 0.1399 - val_loss: 0.9180 - val_acc: 0.4223 - val_mse: 0.1422
Epoch 4/10
97/97 [==============================] - 3s 28ms/step - loss: 0.8800 - acc: 0.4301 - mse: 0.1388 - val_loss: 0.9000 - val_acc: 0.4223 - val_mse: 0.1416
Epoch 5/10
97/97 [==============================] - 3s 28ms/step - loss: 0.8649 - acc: 0.4301 - mse: 0.1384 - val_loss: 0.8875 - val_acc: 0.4223 - val_mse: 0.1413
Epoch 6/10
97/97 [==============================] - 3s 33ms/step - loss: 0.8542 - acc: 0.4301 - mse: 0.1382 - val_loss: 0.8779 - val_acc: 0.4223 - val_mse: 0.1412
Epoch 7/10
97/97 [====

In [23]:
modelsequ2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 200, 10)           2000      
                                                                 
 global_average_pooling1d_1   (None, 10)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 100)               1100      
                                                                 
 Output_layer (Dense)        (None, 4)                 404       
                                                                 
Total params: 3,504
Trainable params: 3,504
Non-traina

In [24]:
modelsequ2.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 0s 5ms/step - loss: 0.8557 - acc: 0.4223 - mse: 0.1410


[0.8557314276695251, 0.4222680330276489, 0.14097967743873596]

In [25]:
pred = modelsequ2.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Vectorize layer int

In [26]:
vectorize_layer_tfidf = TextVectorization(
    ngrams=None, 
    max_tokens=200, vocabulary=None,
    output_mode='int', output_sequence_length=None,
)

vectorize_layer_tfidf.adapt(text_dataset)

In [27]:
modelsequ3 = Sequential()
modelsequ3.add(vectorize_layer_tfidf)
modelsequ3.add(layers.Embedding(vocab_size, embedding_dim, name="embedding"))
modelsequ3.add(layers.GlobalAveragePooling1D())
modelsequ3.add(layers.Dense(50, activation='relu', name="Hidden_layer"))
modelsequ3.add(layers.Dense(4, activation='softmax', name="Output_layer"))
modelsequ3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc', 'mse']) 

In [28]:
# Fit the model using the train and test datasets.
modelsequ3.fit(
    train_dataset.batch(batch_size),
    validation_data=test_dataset.batch(batch_size),
    epochs=epochs)

Epoch 1/10
97/97 [==============================] - 4s 32ms/step - loss: 1.0274 - acc: 0.4293 - mse: 0.1530 - val_loss: 0.9977 - val_acc: 0.4223 - val_mse: 0.1491
Epoch 2/10
97/97 [==============================] - 3s 29ms/step - loss: 0.9428 - acc: 0.4301 - mse: 0.1436 - val_loss: 0.9420 - val_acc: 0.4223 - val_mse: 0.1438
Epoch 3/10
97/97 [==============================] - 3s 27ms/step - loss: 0.9004 - acc: 0.4301 - mse: 0.1400 - val_loss: 0.9133 - val_acc: 0.4223 - val_mse: 0.1420
Epoch 4/10
97/97 [==============================] - 3s 28ms/step - loss: 0.8773 - acc: 0.4301 - mse: 0.1388 - val_loss: 0.8967 - val_acc: 0.4223 - val_mse: 0.1414
Epoch 5/10
97/97 [==============================] - 3s 27ms/step - loss: 0.8636 - acc: 0.4301 - mse: 0.1383 - val_loss: 0.8862 - val_acc: 0.4223 - val_mse: 0.1412
Epoch 6/10
97/97 [==============================] - 3s 33ms/step - loss: 0.8547 - acc: 0.4301 - mse: 0.1381 - val_loss: 0.8786 - val_acc: 0.4223 - val_mse: 0.1411
Epoch 7/10
97/97 [====

In [29]:

modelsequ3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 10)          2000      
                                                                 
 global_average_pooling1d_2   (None, 10)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 Hidden_layer (Dense)        (None, 50)                550       
                                                                 
 Output_layer (Dense)        (None, 4)                 204       
                                                                 
Total params: 2,754
Trainable params: 2,754
Non-traina

In [30]:
modelsequ3.evaluate(sentences_test,y_test_categorical)

76/76 [==============================] - 1s 3ms/step - loss: 0.8612 - acc: 0.4223 - mse: 0.1409


[0.8612480759620667, 0.4222680330276489, 0.14091575145721436]

In [31]:
pred = modelsequ3.predict(sentences_test, batch_size=100, verbose=1) 
predicted = np.argmax(pred, axis=1)  
report = classification_report(np.argmax(y_test_categorical, axis=1), predicted)  
print(report)

25/25 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       469
           1       0.00      0.00      0.00       501
           2       0.00      0.00      0.00       431
           3       0.42      1.00      0.59      1024

    accuracy                           0.42      2425
   macro avg       0.11      0.25      0.15      2425
weighted avg       0.18      0.42      0.25      2425



c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\camilo\anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
